In [1]:
from __future__ import print_function
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))
from pkg.controller.robot_config import RobotType
from pkg.utils.test_scripts import *
from pkg.planning.filtering.filter_interface import *
from pkg.planning.constraint.constraint_common import BindingTransform

ROBOT_TYPE = RobotType.panda
VISUALIZE = True

ROBOT_NAME, TOOL_LINK, TOOL_XYZ, TOOL_RPY, HOME_POSE, GRIP_DEPTH = get_single_robot_params(ROBOT_TYPE)
s_builder, pscene = prepare_single_robot_scene(ROBOT_TYPE, ROBOT_NAME, TOOL_LINK, TOOL_XYZ, TOOL_RPY, VISUALIZE=VISUALIZE)
crob, gscene = pscene.combined_robot, pscene.gscene

connection command:
panda0: False
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


## check GraspChecker scene

In [2]:
from pkg.planning.filtering.grasp_filter import GraspChecker
gcheck = GraspChecker(pscene)
gcheck.show_vertices = True

cpath = os.path.join(SCENE_PATH, "GraspChecker")
files = sorted(os.listdir(cpath))
print(files)

OSError: [Errno 2] No such file or directory: '/home/rnb/Projects/rnb-planning/data/checker_scenes/GraspChecker'

In [ ]:
file_path = os.path.join(cpath, files[7])
print("file_path: {}".format(file_path))
scene_data = load_pickle(file_path)
state, btf, Q_dict, kwargs, error_state, result = load_unpack_scene_args(pscene, scene_data)

res = gcheck.check(btf=btf, Q_dict=Q_dict, **kwargs)
print("result/log: {} / {}".format(res, result))

## check ReachChecker scene

In [ ]:
from pkg.planning.filtering.reach_filter import ReachChecker
rcheck = ReachChecker(pscene)

cpath = os.path.join(SCENE_PATH, "ReachChecker")
files = sorted(os.listdir(cpath))
print(files)

In [ ]:
file_path = os.path.join(cpath, files[0])
print("file_path: {}".format(file_path))
scene_data = load_pickle(file_path)
state, btf, Q_dict, kwargs, error_state, result = load_unpack_scene_args(pscene, scene_data)

res = rcheck.check(btf=btf, Q_dict=Q_dict, **kwargs)
print("result/log: {} / {}".format(res, result))

## check LatticedChecker scene

In [2]:
from pkg.planning.filtering.grasp_filter import GraspChecker
gcheck = GraspChecker(pscene)

from pkg.planning.filtering.latticized_filter import LatticedChecker
lcheck = LatticedChecker(pscene, gcheck=gcheck)

cpath = os.path.join(SCENE_PATH, "LatticedChecker")
files = sorted(os.listdir(cpath))
print("File count= {}".format(len(files)))

from pkg.planning.filtering.latticized_filter import ARM_SHAPE, GRASP_SHAPE, \
    WDH_GRASP, L_CELL_GRASP, OFFSET_ZERO_GRASP, WDH_ARM, L_CELL_ARM, OFFSET_ZERO_ARM
from pkg.planning.filtering.lattice_model.latticizer_py import get_centers

centers_grasp = get_centers(GRASP_SHAPE, L_CELL_GRASP, OFFSET_ZERO_GRASP)
centers_arm = get_centers(ARM_SHAPE, L_CELL_ARM, OFFSET_ZERO_ARM)

assert len(pscene.robot_chain_dict.values())==1, "not single robot scene"
shoulder_height = lcheck.shoulder_height_dict[ROBOT_NAME]

File count= 899


In [3]:
file_path = os.path.join(cpath, files[0])
print("file_path: {}".format(file_path))
scene_data = load_pickle(file_path)
state, btf, Q_dict, kwargs, error_state, result = load_unpack_scene_args(pscene, scene_data)
res = lcheck.check(btf=btf, Q_dict=Q_dict, **kwargs)
print("result/log: {} / {}".format(res, result[1]>0.5))

file_path: /home/rnb/Projects/rnb-planning/data/checker_scenes/LatticedChecker/00000012-OK.pkl
result/log: True / True


In [4]:
gtimer = GlobalTimer.instance()
gtimer.reset()
res_saved_list = []
res_infer_list = []
for fname in files:
    file_path = os.path.join(cpath, fname)
    scene_data = load_pickle(file_path)
    state, btf, Q_dict, kwargs, error_state, result = load_unpack_scene_args(pscene, scene_data)
    with gtimer.block("test"):
        res = lcheck.check(btf=btf, Q_dict=Q_dict, **kwargs)
    res_saved_list.append(result[1]>0.5)
    res_infer_list.append(res)

In [5]:
print("="*50)
print_confusion_mat(GT=res_saved_list, Res=res_infer_list)
print("="*50)
print(gtimer)
print("="*50)

           PP         PN        899
GP        370          0    100.00%
GN          0        529    100.00%
AL    100.00%    100.00%    100.00%
test: 	35818.7 ms/899 = 39.8 ms (27.225/74.495)
get_grasping_vert_infos: 	443.1 ms/899 = 0.5 ms (0.278/1.388)
transform_verts: 	335.3 ms/899 = 0.4 ms (0.202/1.105)
convert_vertices: 	21598.6 ms/899 = 24.0 ms (12.468/56.729)
indexing_vertices: 	646.3 ms/899 = 0.7 ms (0.333/2.693)
query_wait_response: 	12626.6 ms/899 = 14.0 ms (9.938/35.228)



In [6]:
grasp_img = lcheck.grasp_img_p_dict[ROBOT_TYPE.name][0]
grasp_tool_img = grasp_img[:,:,:,0]
grasp_obj_img = grasp_img[:,:,:,1]
grasp_tar_img = grasp_img[:,:,:,2]
arm_img = lcheck.arm_img_p_dict[ROBOT_TYPE.name][0][:,:,:,0]
rh_vals = lcheck.rh_vals_p_dict[ROBOT_TYPE.name][0]

grasp_tool_idx = np.where(grasp_tool_img)
grasp_obj_idx = np.where(grasp_obj_img)
grasp_tar_idx = np.where(grasp_tar_img)
arm_tar_idx = np.where(arm_img)
radius, theta, height = lcheck.rth_last
offset_ee = cyl2cart(radius, theta, height)
print("r, h match: {}".format(np.equal(rh_vals, [radius, height])))

gscene.clear_highlight()
for i_c, center in enumerate(centers_arm[arm_tar_idx]):
    center_rot = np.matmul(Rot_axis(3, theta), center) + [0,0,shoulder_height]
    gscene.add_highlight_axis("arm", "%05d"%(i_c), "base_link", dims=(0.03,) * 3,
                              center=center_rot, orientation_mat=np.identity(3),
                              color=(0.5, 0.5, 0.5, 0.7), axis=None)
    
for i_c, center in enumerate(centers_grasp[grasp_tool_idx]):
    center_rot = np.matmul(Rot_axis(3, theta), center) + offset_ee
    gscene.add_highlight_axis("tool", "%05d"%(i_c), "base_link", dims=(0.02,) * 3,
                              center=center_rot, orientation_mat=np.identity(3),
                              color=(0, 1, 0, 0.7), axis=None)

for i_c, center in enumerate(centers_grasp[grasp_tar_idx]):
    center_rot = np.matmul(Rot_axis(3, theta), center) + offset_ee
    gscene.add_highlight_axis("tar", "%05d"%(i_c), "base_link", dims=(0.02,) * 3,
                              center=center_rot, orientation_mat=np.identity(3),
                              color=(1, 0, 0, 0.7), axis=None)
    
for i_c, center in enumerate(centers_grasp[grasp_obj_idx]):
    center_rot = np.matmul(Rot_axis(3, theta), center) + offset_ee
    gscene.add_highlight_axis("obj", "%05d"%(i_c), "base_link", dims=(0.02,) * 3,
                              center=center_rot, orientation_mat=np.identity(3),
                              color=(1, 1, 0, 0.7), axis=None)
    
pscene.show_binding(btf)

r, h match: [ True  True]


### find idx that deals with object other than obj_0

In [ ]:
for file_idx in range(1173):
    scene_data = load_pickle(os.path.join(SCENE_PATH, "{:08d}-OK.pkl".format(file_idx)))
    global_log = scene_data["global_log"]
#     if global_log["subject"] != "obj_0" and global_log["actor"] != "grip0":
    if global_log["actor"] != "grip0":
        print(file_idx)